In [ ]:
import pandas as pd
from plotly.colors import sequential
from src.db_helpers import get_cases_by_date
from src.plotting_helpers import get_color_for_val
from src.env_helpers import get_commit_url, get_commit_msg, get_deploy_date, get_commit_hash
import ipyvuetify as v
from ipyleaflet import Map, GeoJSON, basemaps, WidgetControl
from ipywidgets import HTML

pd.options.plotting.backend = 'plotly'

cases, date = get_cases_by_date()

In [ ]:
def get_color(val):
    if val in cases.index:
        return get_color_for_val(cases[val], 0, cases.max(), sequential.YlOrRd), 0.8
    else:
        return 'white', 0.0

In [ ]:
commit_url = get_commit_url()
commit_hash = get_commit_hash()
commit_msg = get_commit_msg()
deploy_date = get_deploy_date()

In [ ]:
from pathlib import Path
import json 
json_file = Path.cwd().joinpath('./data/Census_ZIP.geojson')
if json_file.exists():
    with open(str(json_file), 'r') as file:
        zips = json.loads(file.read())
else:
    import urllib.request
    with urllib.request.urlopen('https://opendata.arcgis.com/datasets/41c3a7bd375547069a78fce90153cbc0_5.geojson') as url:
        zips = json.loads(url.read().decode())
    with open(str(json_file), 'w') as file:
        json.dump(zips, file)

In [ ]:
def style_zip(feature):
    color, opacity = get_color(int(feature['properties']['ZIP']))
    return {
        'weight': 0,
        'color': 'white',
        'fillOpacity': opacity,
        'fillColor': color,
    }

In [ ]:
m = Map(basemap=basemaps.OpenStreetMap.BlackAndWhite, center=(33.0,-116.7),zoom=9)

zips_layer = GeoJSON(data=zips,
                     hover_style={'color': 'black', 'opacity': 0.9, 'weight': 1.0},
                     style_callback=style_zip)
m.add_layer(zips_layer)

html = HTML('''Move the cursor over an area''')
html.layout.margin = '0px 20px 20px 20px'
control = WidgetControl(widget=html, position='topright')
m.add_control(control)

def update_html(feature,  **kwargs):
    zip_code = feature['properties']['ZIP']
    cases_in_zip = 0
    if zip_code in cases:
        cases_in_zip = cases[zip_code]
    html.value = '''
        <strong>{}</strong><br>{}<br>Cases: {:.0f}
    '''.format(zip_code,
               feature['properties']['COMMUNITY'],
               cases_in_zip)

zips_layer.on_hover(update_html)

In [ ]:
from src.data_helpers import get_population_per_zip, add_cases_per_100k_col
from src.db_helpers import get_cases_per_week_by_zip
from src.plots import plot_cases_per_week_by_zip
import plotly.graph_objects as go

cases_per_week = get_cases_per_week_by_zip()
cases_per_week = add_cases_per_100k_col(cases_per_week)
population_per_zip = get_population_per_zip()

available_zips = sorted(list(cases_per_week['zip'].unique()))

zip_combo = v.Autocomplete(multiple=True, clearable=True, items=available_zips, v_model=None, 
           hint='Select one or multiple ZIP codes to compare to San Diego\'s average cases per week.', persistent_hint=True)

fig = go.FigureWidget(plot_cases_per_week_by_zip(cases_per_week, population_per_zip, zip_combo.v_model, title=None))

def redraw(widget, event, data):
    global fig
    fig = plot_cases_per_week_by_zip(cases_per_week, population_per_zip, widget.v_model, title=None, fig=fig)
    
zip_combo.on_event('change', redraw)

weekly_cases_ui = v.Container(fluid=True, class_='fill-height', children=[
                v.Row(align='center', justify='center', children=[
                    v.Col(cols=10, children=[
                        v.Card(children=[
                            v.CardTitle(children=['Weekly Covid-19 Cases per 100k Residents by San Diego County ZIP Code']),
                            v.CardText(children=[zip_combo]),
                            fig
                        ])
                    ])
                ])
            ])

In [ ]:
about_data_card = v.Card(children=[
    v.CardTitle(children=['About the Data']),
    v.CardText(children=['This section informs about the data sources this dashboard uses. All data are publicly accessible.']),
    v.Container(fluid=True,children=[
        v.Row(dense=True,children=[
            v.Col(cols=12,children=[
                v.Card(children=[
                    v.CardTitle(children=['Covid-19 Case Data']),
                    v.CardText(children=[
                        v.Html(tag='p', children=[
                            'All Covid-19 case data have been received from the Internet Archive. '
                            'The original data have been prepared in PDF documents with the title "Daily Coronavirus '
                            'Disease 2019 (COVID-19) Summary of Cases by Zip Code of Residence" '
                            'by the County of San Diego, Health and Human Services Agency, Public Health Services, '
                            'Epidemiology and Immunization Services Branch.'
                        ]),
                        v.Html(tag='p', children=['Data are usually updated from the Internet Archive once per day.']),
                        v.Html(tag='p', children=['Original data source: San Diego County Communicable Disease Registry, '
                                                  'SANDAG 2019 Population Estimates (Prepared June 2020).']),
                        v.Html(tag='p', children=['Zip code is zip code of residence, which may not be location of exposure. '
                                                   'Rates based on small case counts may vary considerably and should be interpreted with caution.',
                                                   'Data are preliminary and subject to change. Total population=3,351,784.'])
                    ]),
                    v.CardActions(children=[
                        v.Btn(text=True,
                              color='blue',
                              children=[
                                  v.Html(tag='a',
                                         attributes={'href':'https://web.archive.org/web/*/https://www.sandiegocounty.gov/content/dam/sdc/hhsa/programs/phs/Epidemiology/COVID-19%20Summary%20of%20Cases%20by%20Zip%20Code.pdf', 'target': '_blank'},
                                         children=['Data Source'])
                        ]),
                        v.Btn(text=True,
                              color='blue',
                              children=[#https://archive.org/donate/
                                  v.Html(tag='a',
                                         attributes={'href':'https://www.sandiegocounty.gov/content/sdc/hhsa/programs/phs/community_epidemiology/dc/2019-nCoV/status.html', 'target': '_blank'},
                                         children=['County of San Diego'])
                        ]),
                        v.Btn(text=True,
                              color='blue',
                              children=[#
                                  v.Html(tag='a',
                                         attributes={'href':'https://archive.org/donate/', 'target': '_blank'},
                                         children=['Donate to the Internet Archive'])
                        ])
                    ]),
                ]),
            ]),
            v.Col(cols=6,children=[
                v.Card(children=[
                        v.CardTitle(children=['ZIP Code Boundaries']),
                        v.CardText(children=['The ZIP code boundaries shown in all maps are taken from the SANDAG/SanGIS Open Data Portal.']),
                        v.CardActions(children=[
                            v.Btn(text=True,
                                  color='blue',
                                  children=[
                                      v.Html(tag='a',
                                             attributes={'href':'https://sdgis-sandag.opendata.arcgis.com/datasets/zip-code', 'target': '_blank'},
                                             children=['Data Source'])
                            ]),
                            v.Btn(text=True,
                                  color='blue',
                                  children=[
                                      v.Html(tag='a',
                                             attributes={'href':'https://sandag.org/resources/maps_and_gis/gis_downloads/disclaimer.asp', 'target': '_blank'},
                                             children=['SANDAG Disclaimer'])
                            ])
                        ]),
                    ]),
                ]),

            v.Col(cols=6,children=[
                v.Card(children=[
                    v.CardTitle(children=['Demographic Data']),
                    v.CardText(children=[
                        v.Html(tag='p',children=['Population counts and other demographic data have been received from the SANDAG Data Surfer.']),
                        v.Html(tag='i',children=['SANDAG, Current Estimates (data extracted on: 9/2020)'])
                    ]),
                    v.CardActions(children=[
                        v.Btn(text=True,
                              color='blue',
                              children=[
                                  v.Html(tag='a',
                                         attributes={'href':'https://datasurfer.sandag.org/dataoverview', 'target': '_blank'},
                                         children=['Data Source'])
                        ]),
                    ])
                ]),
            ])
        ])
    ])
])


data_info_ui = v.Container(fluid=True, class_='fill-height', children=[
                v.Row(align='center', justify='center', children=[
                    v.Col(cols=10, children=[
                        about_data_card
                    ])
                ])
            ])

In [ ]:
map_ui = v.Container(fluid=True, class_='fill-height', children=[
                v.Row(align='center', justify='center', children=[
                    v.Col(cols=10, children=[
                        v.Card(children=[
                            v.CardTitle(children=['Covid-19 Cases on {} by ZIP Code'.format(date)]),
                            #v.CardText(children=['This map shows '])
                            m
                        ])
                    ])
                ])
            ])

ui = [
    v.AppBar(app=True, dark=True, absolute=True, color='blue darken-2', children=[
        v.ToolbarTitle(_metadata={'mount_id': 'content-bar'},
              children=['San Diego Covid-19 Dashboard']),
        
    ]),
    v.Content(children=[
        v.Tabs(align_with_title=False, children=[
            v.Tab(children=['Map']),
            v.Tab(children=['Timeline per ZIP']),
            v.Tab(children=['About the Data']),
            v.TabItem(children=[
                map_ui
            ]),
            v.TabItem(children=[
                weekly_cases_ui
            ]),
            v.TabItem(children=[
                data_info_ui
            ])
        ]),
    ])
]

footer_content = []
if commit_url and commit_msg and commit_hash and deploy_date:
    footer_content.extend(
    'Deployed on {:%Y-%m-%d %H:%M} in Kubernetes (Commit '.format(deploy_date),
     v.Html(tag='a', attributes=dict(href=commit_url,target='_blank'), children=[commit_hash[:7]]),
    ':',
    v.Html(tag='i', children=[commit_msg]),
    ') – '
    )

footer_content.extend(
    'Questions or ideas? Please share your ',
    v.Html(tag='a', attributes=dict(href='https://survey.survicate.com/a8285d0a4afe7aa3/',target='_blank', children=['feedback'])),
    '!'
)

ui.append(v.Footer(absolute=True, children=[
            v.Col(cols=12, children=[
                v.Html(tag='small', children=footer_content)
            ])
        ]))

v.Layout(_metadata={'mount_id': 'content'}, children=ui)
